# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, Model, Webservice
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep

import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
We will be using the 'Credit Card customers' dataset from Kaggle and predicting if a customer is likely to churn.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# Choose a name for experiment
experiment_name = 'demo-automl-experiment'

experiment=Experiment(ws, experiment_name)

# Prepare the datastore to upload data
datastore = ws.get_default_datastore()

# Set the upload location to target_path in datastore
datastore.upload(src_dir='./', target_path = 'data')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E8YPJSNN8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Uploading an estimated of 9 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 9
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 9
Uploading ./automl.ipynb.amltmp
Uploaded ./automl.ipynb.amltmp, 3 files out of an estimated total of 9
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 4 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 5 files out of an estimated total of 9
Uploading ./score.py
Uploaded ./score.py, 6 files out of an estimated total of 9
Uploading ./train.py
Uploaded ./train.p

$AZUREML_DATAREFERENCE_56a25c2972504a6e8e4c55ba3e3e1f57

In [3]:
# Choose a name for CPU cluster
compute_name = "demo-cluster"

# Verify that cluster does not already exist
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
dataset = Dataset.get_by_name(ws, name = "bankchurners")
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/02-08-2021_115114_UTC/BankChurners.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "e82585d3-5310-4452-a65a-033e2142d090",
    "name": "bankchurners",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-138143', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-138143')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# List the automl settings

automl_settings = {"featurization":"auto",
                   "primary_metric":"accuracy",
                   "enable_early_stopping":True,
                   "max_concurrent_iterations":5,
                   "enable_onnx_compatible_models":True,
                   "n_cross_validations": 3
}

# List the automl config

automl_config = AutoMLConfig(compute_target= "demo-cluster",
                             task="classification",
                             training_data=dataset,
                             label_column_name="Attrition_Flag",
                             experiment_timeout_minutes=20,
                             **automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

# Submit the experiment
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [7]:
remote_run.wait_for_completion(show_output = True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1627                             |Attrited Customer                |10127                                 |
+---------------------------------+----------

{'runId': 'AutoML_d81a2ede-8205-4c13-870a-1cc62b1d5b00',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-08T11:57:00.994289Z',
 'endTimeUtc': '2021-02-08T12:14:47.029299Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'demo-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"demo-automl-experiment","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-138143","workspace_name":"quick-starts-ws-138143","region":"southcentralus","compute_target":"demo-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Retrieve the best AutoML model
best_run, best_model = remote_run.get_output()
best_metrics = best_run.get_metrics()

# log parameters of all the estimators of the best AutoML Model
best_model.steps[1][1].estimators

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


[('0',
  Pipeline(memory=None,
           steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                  ('lightgbmclassifier',
                   LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split', learning_rate=0.1,
                                      max_depth=-1, min_child_samples=20,
                                      min_child_weight=0.001, min_split_gain=0.0,
                                      n_estimators=100, n_jobs=1, num_leaves=31,
                                      objective=None, random_state=None,
                                      reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0, verbose=-10))],
           verbose=False)),
 ('31',
  Pipeline(memory=None,
           steps=[('sparsenormalizer',
    

In [9]:
#Save the best model
model = best_run.register_model(model_name='best-automl-model',model_path='outputs/model.pkl', tags={'Method':'AutoML'}, properties={'accuracy': best_metrics['accuracy']})
print(model)
print(best_run)

Model(workspace=Workspace.create(name='quick-starts-ws-138143', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-138143'), name=best-automl-model, id=best-automl-model:1, version=1, tags={'Method': 'AutoML'}, properties={'accuracy': '0.9704749575800129'})
Run(Experiment: demo-automl-experiment,
Id: AutoML_d81a2ede-8205-4c13-870a-1cc62b1d5b00_38,
Type: azureml.scriptrun,
Status: Completed)


In [10]:
print(best_metrics["accuracy"])
print(best_model._final_estimator)

0.9704749575800129
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.automl.core.shared import constants

best_run.download_file('./outputs/model.pkl', './outputs/bankchurners-auto-model.pkl')
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', './outputs/score_aml.py')
best_run.download_file('./automl_driver.py', './outputs/automl_driver.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

# Create the inference config

inference_config = InferenceConfig(entry_script='./score.py', environment=myenv)


In [12]:
f = open("envnew.yml", "w")
f.write(myenv.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", myenv.python.conda_dependencies.serialize_to_string())


packages # Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.21.0
  - inference-schema
  - azureml-interpret==1.21.0
  - azureml-defaults==1.21.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



In [13]:
# Deploy the model as a webservice
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb = 1,
    enable_app_insights = True
)
service = Model.deploy(ws, 'model-service', [model], inference_config, deployment_config)

print(service.state)

# Wait until deployment is complete
service.wait_for_deployment(show_output = True)


Transitioning
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-138143', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-138143'), name=model-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://af1ec328-1599-4822-b329-47a1c31efa60.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})>


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import json

# Importing the data set for testing 

sample_data = dataset.to_pandas_dataframe().sample(10)
data_sample = sample_data.drop("Attrition_Flag", axis=1)
test_sample = json.dumps({"data": data_sample.to_dict()})

print(test_sample)

{"data": {"CLIENTNUM": {"390": 715022133, "224": 719662458, "7699": 713772333, "4255": 714119283, "2587": 711639708, "6916": 717862308, "8531": 771935283, "9391": 709410033, "9941": 716464533, "3898": 721202058}, "Customer_Age": {"390": 53, "224": 44, "7699": 45, "4255": 42, "2587": 42, "6916": 45, "8531": 47, "9391": 56, "9941": 32, "3898": 62}, "Gender": {"390": "M", "224": "F", "7699": "F", "4255": "F", "2587": "M", "6916": "F", "8531": "F", "9391": "M", "9941": "F", "3898": "F"}, "Dependent_count": {"390": 2, "224": 2, "7699": 3, "4255": 2, "2587": 3, "6916": 2, "8531": 4, "9391": 2, "9941": 1, "3898": 0}, "Education_Level": {"390": "Graduate", "224": "Graduate", "7699": "Unknown", "4255": "Unknown", "2587": "Graduate", "6916": "College", "8531": "Graduate", "9391": "High School", "9941": "Graduate", "3898": "High School"}, "Marital_Status": {"390": "Married", "224": "Divorced", "7699": "Divorced", "4255": "Single", "2587": "Single", "6916": "Divorced", "8531": "Single", "9391": "U

In [17]:
import requests
scoring_uri = 'http://af1ec328-1599-4822-b329-47a1c31efa60.southcentralus.azurecontainer.io/score'
headers = {'Content-type': 'application/json'}

response = requests.post(scoring_uri, test_sample, headers=headers)

In [18]:
# Print results from the inference
print(response.status_code)
print(response.text)

200
"{\"result\": [\"Existing Customer\", \"Existing Customer\", \"Attrited Customer\", \"Attrited Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\"]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
# Print web service logs
service_logs = service.get_logs()

for line in service_logs.split('\n'):
    print(line)

# Delete the service
service.delete()

2021-02-08T12:51:31,339552900+00:00 - nginx/run 
2021-02-08T12:51:31,368879200+00:00 - iot-server/run 
2021-02-08T12:51:31,374213900+00:00 - rsyslog/run 
2021-02-08T12:51:31,398079200+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml